<a href="https://colab.research.google.com/github/Coffinbrain/lessons/blob/main/DL_E01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.layers(x)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP()
model = model.to(device)

In [26]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(16*5*5, 120)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()

        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet()
model = model.to(device)

In [31]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = x.repeat(1, 3, 1, 1)  # Convert 1x28x28 to 3x28x28
        x = nn.functional.interpolate(x, size=(227, 227))  # Resize image to 227x227
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet()
model = model.to(device)

In [5]:
#MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 115546823.09it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 89070363.11it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33210838.34it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17255913.74it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [11]:
#FASHION MNIST
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.FashionMNIST('./data', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

100%|██████████| 26421880/26421880 [00:01<00:00, 16553910.74it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 302571.19it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5510319.01it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 17082497.62it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [32]:
#HWDB1
#参考：https://blog.csdn.net/qq_41684249/article/details/117397215

transform = transforms.Compose([transforms.Resize((32, 32)),transforms.ToTensor(),])

class HWDB(object):
    def __init__(self,path, transform):
        # 预处理过程
        traindir = os.path.join(path, 'train')
        testdir = os.path.join(path, 'test')
        # 读取训练集和测试集
        self.trainset = datasets.ImageFolder(traindir, transform)
        self.testset = datasets.ImageFolder(testdir, transform)
        self.train_size = len(self.trainset) # 训练集大小
        self.test_size = len(self.testset) # 测试集大小
        self.num_classes = len(self.trainset.classes) # 类别数
        self.class_to_idx = self.trainset.class_to_idx # 索引

    def get_sample(self, index=0):
        sample = self.trainset[index]
        sample_img, sample_label = sample # 获取图片和标签
        return sample_img, sample_label

    def get_loader(self, batch_size=100):
        trainloader = DataLoader(self.trainset, batch_size=batch_size, shuffle=True)
        testloader = DataLoader(self.testset, batch_size=batch_size, shuffle=True)
        return trainloader, testloader

hwdb = HWDB(path='/content/drive/MyDrive/HWDB1', transform=transform)
trainloader, testloader = hwdb.get_loader(batch_size=32)


In [33]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # number of epochs
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()




Epoch 1, Loss: 0.025557084009051323
Epoch 2, Loss: 0.00259738159365952
Epoch 3, Loss: 0.006929455325007439
Epoch 4, Loss: 0.24761360883712769
Epoch 5, Loss: 0.3663114905357361
Epoch 6, Loss: 0.0004155081114731729
Epoch 7, Loss: 0.0019783235620707273
Epoch 8, Loss: 0.12799525260925293
Epoch 9, Loss: 0.04716859012842178
Epoch 10, Loss: 0.0051575289107859135


In [ ]:
print(f'MLP_MNIST_Accuracy: {100 * correct // total} %')

MLP_MNIST_Accuracy: 97 %


In [ ]:
print(f'MLP_FASHION_MNIST_Accuracy: {100 * correct // total} %')

MLP_FASHION_MNIST_Accuracy: 96 %


In [23]:
print(f'MLP_HWDB1_Accuracy: {100 * correct // total} %')

MLP_HWDB1_Accuracy: 97 %


In [ ]:
print(f'LeNet_MNIST_Accuracy: {100 * correct // total} %')

LeNet_MNIST_Accuracy: 98 %


In [ ]:
print(f'LeNet_FASHION_MNIST_Accuracy: {100 * correct // total} %')

LeNet_FASHION_MNIST_Accuracy: 98 %


In [29]:
print(f'LeNet_HWDB1_Accuracy: {100 * correct // total} %')

LeNet_HWDB1_Accuracy: 98 %


In [8]:
print(f'AlexNet_MNIST_Accuracy: {100 * correct // total} %')

AlexNet_MNIST_Accuracy: 98 %


In [13]:
print(f'AlexNet_FASHION_MNIST_Accuracy: {100 * correct // total} %')

AlexNet_FASHION_MNIST_Accuracy: 98 %


In [34]:
print(f'AlexNet_HWDB1_Accuracy: {100 * correct // total} %')

AlexNet_HWDB1_Accuracy: 98 %
